In [41]:
import pandas as pd
import os
import numpy as np

filenames = os.listdir('./team_stats/')
filenames_stats = filenames[1::2]
filenames_logs = filenames[0::2]
filenames_stats[0], filenames_logs[0]

('49ers_stats.csv', '49ers_logs.csv')

In [7]:
df_log = pd.read_csv(f'./team_stats/{filenames_logs[0]}')
df_log

,Week,Day,Date,Unnamed: 3,Unnamed: 4,OT,Unnamed: 6,Opp,Tm,Opp.1,...,FGA,XPM,XPA,Pnt,Yds.3,3DConv,3DAtt,4DConv,4DAtt,ToP
0,1,Sun,September 11,boxscore,L,NaN,@,Chicago Bears,10,19,...,1,1,1,4,161,8,17,0,2,33:28
1,2,Sun,September 18,boxscore,W,NaN,NaN,Seattle Seahawks,27,7,...,3,3,3,3,132,6,15,2,3,38:20
2,3,Sun,September 25,boxscore,L,NaN,@,Denver Broncos,10,11,...,1,1,1,7,362,1,10,0,0,25:31
3,4,Mon,October 3,boxscore,W,NaN,NaN,Los Angeles Rams,24,9,...,2,3,3,4,158,5,12,0,1,25:26
4,5,Sun,October 9,boxscore,W,NaN,@,Carolina Panthers,37,15,...,2,4,5,2,75,7,12,0,0,30:18
5,6,Sun,October 16,boxscore,L,NaN,@,Atlanta Falcons,14,28,...,0,2,2,3,158,3,8,0,1,26:35
6,7,Sun,October 23,boxscore,L,NaN,NaN,Kansas City Chiefs,23,44,...,3,2,2,1,29,8,14,0,0,33:45
7,8,Sun,October 30,boxscore,W,NaN,@,Los Angeles Rams,31,14,...,1,4,4,3,123,5,9,0,0,29:56
8,10,Sun,November 13,boxscore,W,NaN,NaN,Los Angeles Chargers,22,16,...,3,1,2,4,132,9,17,1,1,36:59
9,11,Mon,November 21,boxscore,W,NaN,@,Arizona Cardinals,38,10,...,1,5,5,4,215,6,11,0,0,29:09


In [8]:
drop_columns_stats = ['Day', 'Date', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 8', 'OT', 'PassY', 'RushY',
        'Opp', 'Tm', 'Opp.1', 'TotYd', ]


In [9]:
drop_columns_logs = [
    'Unnamed: 3', 'Unnamed: 4', 'OT', 'XPM', 'XPA', 'Pnt','Yds.3'
]

In [28]:
rename_dict_logs = {
    'Week':'week',
    'Day':'day',
    'Date':'date',
    'Unnamed: 6':'home/away',
    'Opp':'opponent',
    'Tm':'score_team',
    'Opp.1':'score_opp',
    'Cmp':'pass_cmp_off',
    'Att':'pass_att_off',
    'Yds':'pass_yds_off',
    'TD':'pass_tds_off',
    'Int':'ints_off',
    'Sk':'sacks_off',
    'Yds.1':'sacks_yds_off',
    'Y/A':'pass_yds_per_att',
    'NY/A':'pass_net_yds_per_att',
    'Cmp%':'pass_cmp_prcnt',
    'Rate':'pass_qb_rate',
    'Att.1':'rush_att_off',
    'Yds.2':'rush_yds_off',
    'Y/A.1':'rush_yds_per_att',
    'TD.1':'rush_tds_off',
    'FGM':'fg_made',
    'FGA':'fg_att',
    '3DConv':'3d_made',
    '3DAtt':'3d_att',
    '4DConv':'4d_made',
    '4DAtt':'4d_att',
    'ToP': 'time_poss'
}

In [11]:
df_log_1 = df_log.drop(drop_columns_logs, axis=1)
df_log_1 = df_log_1.rename(columns = rename_dict_logs)
df_log_1.columns

Index(['week', 'day', 'date', 'home_away', 'opponent', 'score_team',
       'score_opp', 'pass_cmp_off', 'pass_att_off', 'pass_yds_off',
       'pass_tds_off', 'ints_off', 'sacks_off', 'sacks_yds_off',
       'pass_yds_per_att', 'pass_net_yds_per_att', 'pass_cmp_prcnt',
       'pass_qb_rate', 'rush_att_off', 'rush_yds_off', 'rush_yds_per_att',
       'rush_tds_off', 'fg_made', 'fg_att', '3d_made', '3d_att', '4d_made',
       '4d_att', 'time_poss'],
      dtype='object')

In [34]:
rename_dict_stats = {
    'Week':'wk',
    'Rec':'record',
    '1stD':'1st_down_off',
    'TO':'turnover_off',
    '1stD.1':'1st_down_def',
    'TotYd.1':'tot_yds_def',
    'PassY.1':'pass_yds_def',
    'RushY.1':'rush_yds_def',
    'TO.1':'turnover_def',
    'Offense':'exp_off',
    'Defense':'exp_def',
    'Sp. Tms':'exp_st',
}

In [35]:
def get_trimmed_df(filename, drop_columns, rename_dict):
    # team = filename.split('_')[0]
    # print(team)
    df = pd.read_csv(f'./team_stats/{filename}')
    df = df.drop(drop_columns, axis=1)
    df = df.rename(columns=rename_dict)
    return df

In [36]:
def combine_trimmed_stats_w_logs(df_stats, df_logs):
    # if team_stats != team_logs:
    #     print('TEAMS DONT MATCH')
    #     return
    combo_df = pd.concat([df_logs, df_stats], axis=1,)
    return combo_df


In [78]:

def get_home_and_away_columns(df):
    df['home/away'] = np.where(
        df['home/away'] == '@', 
        'away', 
        'home'
    )
    
    return df

def get_short_team_name(full_team_name):
    # print(full_team_name)
    short_name = full_team_name.rsplit(None, 1)[-1]
    lc_short_name = short_name.lower()
    return lc_short_name

In [74]:
def clean_and_concat_stats_logs(team):
    stats_filename = f'{team}_stats.csv'
    logs_filename = f'{team}_logs.csv'
    
    # get stats df
    df_stats = get_trimmed_df(stats_filename, drop_columns_stats, rename_dict_stats)
    
    # get logs df
    df_logs = get_trimmed_df(logs_filename, drop_columns_logs, rename_dict_logs)
    
    # combine dfs using concat
    df_combo = combine_trimmed_stats_w_logs(df_stats, df_logs)
    
    # set home/away columns replacing NaN and @ with home and away
    df_combo = get_home_and_away_columns(df_combo)
    
    # set opponent column to team name only, remove city
    df_combo['opponent'] = df_combo['opponent'].apply(lambda x: get_short_team_name(x))
    
    # check that weeks in both dataframes match after join
    wks_equal = df_combo['week'].equals(df_combo['wk'])
    if wks_equal != True:
        print('ERROR: weeks did not match in join')
        return 
    # weeks match so drop one of the weeks columns
    df_combo.drop(['wk'], axis=1)
    
    # create team column
    df_combo['team'] = team
    
    # save df to csv file
    df_combo.to_csv(f'./team_stats_combo/{team}_combo_stats.csv')
    
    return df_combo

In [80]:
df_texans = clean_and_concat_stats_logs('texans')

In [81]:
df_texans

,week,day,date,home/away,opponent,score_team,score_opp,pass_cmp_off,pass_att_off,pass_yds_off,...,turnover_off,1st_down_def,tot_yds_def,pass_yds_def,rush_yds_def,turnover_def,exp_off,exp_def,exp_st,team
0,1,Sun,September 11,home,colts,20,20,23,37,222,...,1.0,33,517,340,177,2.0,-2.14,-5.67,9.43,texans
1,2,Sun,September 18,away,broncos,9,16,19,38,154,...,NaN,20,350,201,149,1.0,-12.19,-3.38,5.20,texans
2,3,Sun,September 25,away,bears,20,23,20,32,237,...,2.0,17,363,82,281,2.0,-1.75,-2.69,1.18,texans
3,4,Sun,October 2,home,chargers,24,34,26,35,215,...,2.0,22,419,338,81,1.0,2.14,-15.96,4.62,texans
4,5,Sun,October 9,away,jaguars,13,6,16,24,132,...,NaN,22,422,286,136,2.0,-2.00,2.39,3.89,texans
5,7,Sun,October 23,away,raiders,20,38,28,41,296,...,1.0,25,400,236,164,NaN,5.28,-23.48,1.03,texans
6,8,Sun,October 30,home,titans,10,17,17,30,118,...,1.0,16,354,40,314,2.0,-14.53,-2.29,7.78,texans
7,9,Thu,November 3,home,eagles,17,29,13,22,135,...,2.0,24,360,217,143,1.0,1.89,-17.84,2.38,texans
8,10,Sun,November 13,away,giants,16,24,22,37,286,...,2.0,19,367,176,191,NaN,0.40,-10.29,0.41,texans
9,11,Sun,November 20,home,commanders,10,23,19,33,127,...,2.0,20,344,191,153,NaN,-15.79,-4.31,2.12,texans


In [61]:
from team_names import team_names
print(team_names)

['patriots', 'cowboys', 'giants', 'bills', 'lions', 'texans', 'ravens', 'lions', 'vikings', 'eagles', 'falcons', 'bears', '49ers', 'rams', 'bengals', 'raiders', 'buccaneers', 'browns', 'broncos', 'jaguars', 'jets', 'commanders', 'titans', 'steelers', 'packers', 'dolphins', 'seahawks', 'chiefs', 'chargers', 'colts', 'saints', 'panthers', 'cardinals']


In [84]:
for team in team_names[8:]:
    print(team)
    clean_and_concat_stats_logs(team)

vikings
eagles
falcons
bears
49ers
rams
bengals
raiders
buccaneers
browns
broncos
jaguars
jets
commanders
titans
steelers
packers
dolphins
seahawks
chiefs
chargers
colts
saints
panthers
cardinals
